In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive/Colab\ Notebooks/Synset_Grouping

/content/drive/.shortcut-targets-by-id/1GSBPdsf5hLXUWvkP6mXrY40FQGH44Er5/Synset_Grouping


In [ ]:
ls

1_prepare_synset_data.ipynb
2_prepare_synset_data_part2.ipynb
3_group_synset.ipynb
4_label_cluster_title_to_topic4.ipynb
cluster_matrix_topic4_5000sents.csv
preprocessed_topic4_corpus.csv
topic4_dataset.csv
topic4_sent_corpus.csv
topic4_verb_cluster_data_5000sents.csv
topic4_verb_list_5000sents.csv
vp_synset_data_topic4_5000sents.csv
wrong_4_label_cluster_title_to_topic4.ipynb


Created files: 
- 'vp_list.csv' : VP, Main Verb, Lemmatized Verb
- 'lemma_list.csv' : Lemmatized Verb, Frequency, Synset, Status

In [ ]:
import time

start_time = time.time()

In [ ]:
import spacy   
import numpy as np
import pandas as pd
from spacy.matcher import Matcher                                                                                                                                                                                         
nlp = spacy.load('en_core_web_sm')  # download model first
from nltk import tokenize

from itertools import chain
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

### Load dataset and tokenize

In [ ]:
data = pd.read_csv('preprocessed_topic4_corpus.csv')
data.head()

,Unnamed: 0,doc_id,sent_id,abstract_sents
0,0,0,0,report retinal vein occlusion rvo age-related ...
1,1,0,1,clinical data including fundus photographs opt...
2,2,0,2,twenty-three eyes 21 patients included median ...
3,3,0,3,twelve eyes 522 submacular hemorrhage 11 478 rvo
4,4,0,4,twelve patients 609 vaccinated pfizer vaccine ...


In [ ]:
data.drop(data[data.isna().any(axis=1)].index, axis=0, inplace=True)

In [ ]:
len(data)

62499

In [ ]:
data.reset_index(inplace = True)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
vp_pattern=[{'POS': 'VERB', 'OP': '?'},{'POS': 'ADV', 'OP': '*'},{'POS': 'AUX', 'OP': '*'}, {'POS': 'VERB', 'OP': '+'}]
vp_matcher = Matcher(nlp.vocab) 
vp_matcher.add("verb-phrases", [vp_pattern], greedy = "LONGEST")

main_verb_pattern = [{'POS': 'VERB', 'OP': '+'}]
main_verb_matcher = Matcher(nlp.vocab) 
main_verb_matcher.add("main-verb", [main_verb_pattern])

In [ ]:
vp_list = []
main_verb_list = []

In [ ]:
# df columns 

# doc_id, sent_id, word_id, word, cluster_title(grouped title/ taken verb)

In [ ]:
sent_num = len(data)

In [ ]:
for idx in range(sent_num):
  sentence = data.loc[idx, 'abstract_sents']
  doc = nlp(sentence) 
  
    # VP extraction 
  vp_matches = vp_matcher(doc) 
  vp_list = [str(doc[start:end]) for _, start, end in vp_matches]

  start_list = [start for _, start, end in vp_matches]

  if vp_list:
    data.at[idx,'verb'] = np.array(vp_list, dtype=object)
    data.at[idx,'verb_id'] = np.array(start_list, dtype=object)
  else:
    data.at[idx,'verb'] = np.nan
    data.at[idx,'verb_id'] = np.nan

In [ ]:
data.dropna(axis=0, inplace=True)

In [ ]:
data.head()

,index,Unnamed: 0,doc_id,sent_id,abstract_sents,verb,verb_id
0,0,0,0,0,report retinal vein occlusion rvo age-related ...,"[report, related, -related, developing, conduc...","[0, 7, 11, 14, 27]"
1,1,1,0,1,clinical data including fundus photographs opt...,"[including, reviewed]","[2, 10]"
2,2,2,0,2,twenty-three eyes 21 patients included median ...,"[included, range]","[6, 13]"
4,4,4,0,4,twelve patients 609 vaccinated pfizer vaccine ...,[vaccinated],[3]
5,5,5,0,5,sixteen patients 762 experienced ocular diseas...,[experienced],[3]


In [ ]:
data = data.explode(['verb', 'verb_id'])

In [ ]:
topic4_verb_df = data[['doc_id', 'sent_id', 'verb_id', 'verb']]

In [ ]:
topic4_verb_df.head()

,doc_id,sent_id,verb_id,verb
0,0,0,0,report
0,0,0,7,related
0,0,0,11,-related
0,0,0,14,developing
0,0,0,27,conducted


In [ ]:
# lemmatized verb

topic4_verb_df['check_verb'] = topic4_verb_df['verb'].apply(lambda x: main_verb_matcher(nlp(x)) if main_verb_matcher(nlp(x)) else np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
topic4_verb_df.head()

,doc_id,sent_id,verb_id,verb,check_verb
0,0,0,0,report,NaN
0,0,0,7,related,"[(9148364968459358513, 0, 1)]"
0,0,0,11,-related,"[(9148364968459358513, 0, 1)]"
0,0,0,14,developing,"[(9148364968459358513, 0, 1)]"
0,0,0,27,conducted,"[(9148364968459358513, 0, 1)]"


In [ ]:
topic4_verb_df.dropna(axis=0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
topic4_verb_df['main_verb'] = topic4_verb_df['verb'].apply(lambda x: [str(nlp(x)[start:end]) for _, start, end in main_verb_matcher(nlp(x))][-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
topic4_verb_df['lemma_verb'] = topic4_verb_df['main_verb'].apply(lambda x: WordNetLemmatizer().lemmatize(x, pos ="v"))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
topic4_verb_df.head()

,doc_id,sent_id,verb_id,verb,check_verb,main_verb,lemma_verb
0,0,0,7,related,"[(9148364968459358513, 0, 1)]",related,relate
0,0,0,11,-related,"[(9148364968459358513, 0, 1)]",-related,-related
0,0,0,14,developing,"[(9148364968459358513, 0, 1)]",developing,develop
0,0,0,27,conducted,"[(9148364968459358513, 0, 1)]",conducted,conduct
1,0,1,2,including,"[(9148364968459358513, 0, 1)]",including,include


In [ ]:
topic4_verb_df.to_csv('preprocessed_topic4_verb_list.csv')

In [ ]:
end_time = time.time()

print('Estimate runtime (total): ', (end_time-start_time)/60, ' minutes.')

Estimate runtime (total):  42.84870324532191  minutes.


In [ ]:
# for 5000 sents
# Estimate runtime (total):  5.037542378902435  minutes.

In [ ]:
# for all
#Estimate runtime (total):  42.84870324532191  minutes.